## Applicative

### Applicative definition

In Haskell, the `Applicative` type class is a powerful abstraction that allows you to apply functions wrapped in a context (like a `Maybe`, `List`, or `IO`) to values that are also wrapped in a context. It builds on the `Functor` type class, and provides a way to handle functions that take multiple arguments in a context-aware manner.

#### Key Components

1. **Functor**: The `Applicative` type class is a subclass of `Functor`. This means that any type that is an instance of `Applicative` must also implement `fmap`.

2. **pure**: This function takes a value and wraps it in the applicative context.

3. **(<*> or ap)**: This operator takes a wrapped function and a wrapped value and applies the function to the value.

#### Type Class Definition

Here’s a simplified definition of the `Applicative` type class:

```haskell
class Functor f => Applicative f where
    pure :: a -> f a
    (<*>) :: f (a -> b) -> f a -> f b
```

#### Example 1: `Maybe`

The `Maybe` type is a common instance of `Applicative`. It represents a computation that might fail.

```haskell
-- Using pure to wrap a value
justFive :: Maybe Int
justFive = pure 5  -- Result: Just 5

-- Applying a function wrapped in Maybe
add :: Int -> Int -> Int
add x y = x + y

maybeAdd :: Maybe (Int -> Int)
maybeAdd = pure add

result :: Maybe Int
result = maybeAdd <*> Just 3 <*> Just 4  -- Result: Just 7
```

#### Example 2: Lists

Lists are also instances of `Applicative`. The `(<*>)` operator applies functions in a list to values in another list, producing all combinations.

```haskell
-- Pure to wrap a value in a list
pureList :: [Int]
pureList = pure 5  -- Result: [5]

-- Using a list of functions
addLists :: [Int -> Int]
addLists = [(+1), (+2)]

-- Combining lists
resultList :: [Int]
resultList = addLists <*> [1, 2, 3]  -- Result: [2,3,4,3,4,5]
```

#### Example 3: `IO`

The `IO` type is also an instance of `Applicative`, allowing you to combine actions.

```haskell
-- Pure to wrap a value in IO
pureIO :: IO Int
pureIO = pure 5  -- Result: an IO action that produces 5

-- Combining IO actions
printSum :: IO ()
printSum = (+) <$> getLine <*> getLine >>= print
```

### Summary

- `Applicative` allows you to apply functions in a context to values in a context.
- It is a generalization of `Functor` and provides more functionality.
- Common instances include `Maybe`, lists, and `IO`.

By using `Applicative`, you can elegantly handle computations that may involve multiple contexts, making your Haskell code more concise and expressive.

### Applicative Laws

1. **Identity Law**
   - This law states that applying the identity function wrapped in the applicative context to a value should yield the same value.

   **Law:**  
   ```haskell
   pure id <*> v = v
   ```

   **Example:**
   ```haskell
   maybeValue :: Maybe Int
   maybeValue = Just 5

   result :: Maybe Int
   result = pure id <*> maybeValue  -- Result: Just 5
   ```

2. **Homomorphism Law**
   - This law states that if you take a value, wrap it in the applicative context, and then apply a pure function to it, you should get the same result as applying the function directly to the value and wrapping the result.

   **Law:**  
   ```haskell
   pure f <*> pure x = pure (f x)
   ```

   **Example:**
   ```haskell
   pureValue :: Maybe Int
   pureValue = pure 3  -- Result: Just 3

   result :: Maybe Int
   result = pure (+1) <*> pure 3  -- Result: Just 4
   ```
3. **Composition Law**
    - This law is meant to ensure that there are no surprises resulting from composing your function applications. Here is the definition of the composition law for applicatives:
    
    **Law**
    ```haskell
    pure (.) <*> u <*> v <*> w = u <*> (v <*> w)
    ```
    **Example:**
    ```haskell
    x :: [Int]
    x = pure (.) <*> [(+1)] <*> [(*2)] <*> [1, 2, 3]
    
    y :: [Int]
    y = [(+1)] <*> ([(*2)] <*> [1, 2, 3])
    
    print (y == x)
    ```
    
4. **Interchange Law**
   - This law states that if you have a value wrapped in an applicative context and a function also wrapped in the same context, you can apply them in either order.

   **Law:**  
   ```haskell
   u <*> pure y = pure ($ y) <*> u
   ```

   **Example:**
   ```haskell
   maybeFunc :: Maybe (Int -> Int)
   maybeFunc = Just (+1)

   result1 :: Maybe Int
   result1 = maybeFunc <*> pure 5  -- Result: Just 6

   result2 :: Maybe Int
   result2 = pure ($ 5) <*> maybeFunc  -- Result: Just 6
   ```

#### Summary of Applicative Laws

- **Identity Law:** The identity function in an applicative context does not change the value.
- **Homomorphism Law:** Applying a pure function to a pure value yields the same result as applying the function directly.
- **Interchange Law:** You can interchange the order of applying a function and a value wrapped in the applicative context.

These laws ensure that `Applicative` instances behave consistently and can be relied upon in functional programming. By adhering to these laws, you can avoid unexpected behaviors and write more predictable code. 

### Exercises: Lookups

In the following exercises you will need to use the following terms to make the expressions typecheck:

1. `pure`
2. `(<$>)`
3. `(<*>)`

Make the following expressions typecheck.

1.
```haskell
added :: Maybe Integer
added = (+3) (lookup 3 $ zip [1, 2, 3] [4, 5, 6])
```

In [9]:
added :: Maybe Integer
added = (+ 3) <$> lookup 3 (zip [1, 2, 3] [4, 5, 6])

2.
```haskell
y :: Maybe Integer
y = lookup 3 $ zip [1, 2, 3] [4, 5, 6]
z :: Maybe Integer
z = lookup 2 $ zip [1, 2, 3] [4, 5, 6]
tupled :: Maybe (Integer, Integer)
tupled = (,) y z
```

In [10]:
y :: Maybe Integer
y = lookup 3 $ zip [1, 2, 3] [4, 5, 6]

z :: Maybe Integer
z = lookup 2 $ zip [1, 2, 3] [4, 5, 6]

tupled :: Maybe (Integer, Integer)
tupled = (,) <$> y <*> z

3.
```haskell
import Data.List (elemIndex)

x :: Maybe Int
x = elemIndex 3 [1, 2, 3, 4, 5]

y :: Maybe Int
y = elemIndex 4 [1, 2, 3, 4, 5]

max' :: Int -> Int -> Int
max' = max

maxed :: Maybe Int
maxed = max' x y
```

In [11]:
import Data.List (elemIndex)

x :: Maybe Int
x = elemIndex 3 [1, 2, 3, 4, 5]

y :: Maybe Int
y = elemIndex 4 [1, 2, 3, 4, 5]

max' :: Int -> Int -> Int
max' = max

maxed :: Maybe Int
maxed = max' <$> x <*> y

4.
```haskell
xs = [1, 2, 3]
ys = [4, 5, 6]

x :: Maybe Integer
x = lookup 3 $ zip xs ys

y :: Maybe Integer
y = lookup 2 $ zip xs ys

summed :: Maybe Integer
summed = sum $ (,) x y
```

In [12]:
xs = [1, 2, 3]
ys = [4, 5, 6]

x :: Maybe Integer
x = lookup 3 $ zip xs ys

y :: Maybe Integer
y = lookup 2 $ zip xs ys

summed = sum <$> ((,) <$> x <*> y)

### Exercise: Identity Instance

Write an Applicative instance for Identity.

```haskell
newtype Identity a = Identity a deriving (Eq, Ord, Show)

instance Functor Identity where
    fmap = undefined

instance Applicative Identity where
    pure = undefined
    (<*>) = undefined
```

In [13]:
newtype Identity a = Identity a deriving (Eq, Ord, Show)

instance Functor Identity where
    fmap f (Identity x) = Identity $ f x

instance Applicative Identity where
    pure = Identity
    (<*>) (Identity f) (Identity x) = Identity $ f x 

### Exercise: Constant Instance

Write an Applicative instance for Constant.

```haskell
newtype Constant a b = Constant { getConstant :: a } deriving (Eq, Ord, Show)

instance Functor (Constant a) where
    fmap = undefined

instance Monoid a => Applicative (Constant a) where
    pure = undefined
    (<*>) = undefined
```

In [14]:
newtype Constant a b = Constant { getConstant :: a } deriving (Eq, Ord, Show)

instance Functor (Constant a) where
    fmap f (Constant x) = Constant x

instance Monoid a => Applicative (Constant a) where
    pure x = Constant mempty
    (<*>) (Constant x1) (Constant x2) = Constant (x1 <> x2)

### Exercise: Fixer Upper

Given the function and values provided, use `(<$>)` from Functor, `(<*>)`
and pure from the Applicative typeclass to fill in missing bits of the
broken code to make it work.

1.
```haskell
const <$> Just "Hello" <*> "World"
```

In [15]:
const <$> Just "Hello" <*> pure "World"

Just "Hello"

2.
```haskell
(,,,) Just 90 <*> Just 10 Just "Tierness" [1, 2, 3]


In [16]:
(,,,) <$> Just 90 <*> Just 10 <*> Just "Tierness" <*> pure [1, 2, 3]

Just (90,10,"Tierness",[1,2,3])

### List Applicative Exercise

Implement the list `Applicative`. Writing a minimally complete `Applicative` instance calls for writing the definitions of both `pure` and `<*>`. We’re going to provide a hint as well. Use the checkers library to validate your `Applicative` instance.

In [ ]:
import Test.QuickCheck
import Test.QuickCheck.Checkers
import Test.QuickCheck.Classes

data List a =
    Nil
  | Cons a (List a)
  deriving (Eq, Show)

instance Functor List where
  fmap :: (a -> b) -> List a -> List b
  fmap _ Nil = Nil
  fmap f (Cons a as) = Cons (f a) (fmap f as)

instance Applicative List where
  pure :: a -> List a
  pure a = Cons a Nil
  (<*>) :: List (a -> b) -> List a -> List b
  Nil <*> _ = Nil
  _ <*> Nil = Nil
  fs <*> xs = flatMap (<$> xs) fs

instance Arbitrary a => Arbitrary (List a) where
  arbitrary = do
    hlist <- arbitrary
    let l = haskellListToList hlist
    return l

instance (Eq a) => EqProp (List a) where
  (=-=) = eq

haskellListToList :: [a] -> List a
haskellListToList = foldr Cons Nil

append :: List a -> List a -> List a
append Nil ys = ys
append xs Nil = xs
append (Cons x xs) ys = Cons x $ xs `append` ys

fold :: (a -> b -> b) -> b -> List a -> b
fold _ b Nil = b
fold f b (Cons h t) = f h (fold f b t)

concat' :: List (List a) -> List a
concat' = fold append Nil

flatMap :: (a -> List b) -> List a -> List b
flatMap f as = concat' $ fmap f as

main :: IO ()
main = do
  let trigger :: List (String, Int, [String])
      trigger = undefined
  quickBatch $ applicative trigger

### ZipList Applicative Exercise
Implement the `ZipList` `Applicative`. Use the `checkers` library to validate
your `Applicative` instance. We’re going to provide the `EqProp` instance
and explain the weirdness in a moment.